## Notebook for playing around with variables in experimental and master objects

In [1]:
%run ./rob_setup_notebook.ipynb

import utils.interareal_analysis as ia
import utils.interareal_plotting as ip
from ia_funcs import *


Suite2p path: ['/home/rlees/Documents/Code/suite2p/suite2p']
QNAP_path: /home/rlees/mnt/qnap 
QNAP_data_path /home/rlees/mnt/qnap/Data 
Pickle folder: /home/rlees/mnt/qnap/pkl_files 
Master pickle: /home/rlees/mnt/qnap/master_pkl/master_obj.pkl 
Figure save path: /home/rlees/mnt/qnap/Analysis/Plots 
STA movie save path: /home/rlees/mnt/qnap/Analysis/STA_movies 
S2 borders path: /home/rlees/mnt/qnap/Analysis/S2_borders


### Add or check attributes in all exp_obj pkl files

In [ ]:
pkl_files = os.listdir(pkl_folder)

for i,pkl in enumerate(pkl_files):
    if '.pkl' in pkl:
        
        pkl_path = os.path.join(pkl_folder, pkl)

        with open(pkl_path, 'rb') as f:
            exp_obj = pickle.load(f)
            
#             with open(pkl_path, 'wb') as f:
#                 pickle.dump(exp_obj, f)
                
#         pkl_list = [exp_obj.photostim_r, exp_obj.photostim_s]

#         if exp_obj.spont.n_frames > 0:
#             pkl_list.append(exp_obj.spont)

#         if exp_obj.whisker_stim.n_frames > 0:
#             pkl_list.append(exp_obj.whisker_stim)
        
#         for sub_obj in pkl_list:

### Play with data from individual experimental objects

In [2]:
# List of pickle object file names
# '2019-06-11_J048',
# '2019-06-13_J052',
# '2019-06-25_RL048',
# '2019-07-15_RL052',
# '2019-08-14_J058',
# '2019-08-14_J059',
# '2019-08-15_RL055',
# '2019-11-14_J061',
# '2019-11-14_J063',
# '2019-11-15_J064',
# '2019-11-18_J065',
# '2020-02-27_RL075',
# '2020-03-03_RL074',
# '2020-03-05_RL070',
# '2020-03-05_RL072',
# '2020-03-12_RL070',
# '2020-03-16_RL074',
# '2020-03-19_RL075',
# '2020-03-20_RL080'

pkl_path = os.path.join(pkl_folder, 'Leave_out', '2020-03-20_RL080.pkl')

with open(pkl_path, 'rb') as f:
    exp_obj = pickle.load(f)

### Show all attributes in exp_obj or sub_objs

In [3]:
# Show all attributes in exp_obj or sub_objs
print('Session object attributes')
for key, value in vars(exp_obj).items():
    print(key)

print('\nExperimental object attributes')
for key, value in vars(exp_obj.photostim_r).items():
    print(key)

Session object attributes
ss_id
sheet_name
s2p_path
photostim_r
photostim_s
whisker_stim
spont
frame_list

Experimental object attributes
tiff_path
naparm_path
paq_path
sheet_name
s2p_path
stim_channel
stim_type
fps
frame_x
frame_y
n_planes
pix_sz_x
pix_sz_y
scan_x
scan_y
zoom
n_frames
last_good_frame
n_groups
n_reps
n_shots
n_trials
inter_point_delay
spiral_size
single_stim_dur
stim_dur
duration_frames
stim_start_frames
frames
cell_id
n_units
cell_plane
cell_med
cell_s1
cell_x
cell_y
raw
mean_img
xoff
yoff
pre_frames
post_frames
all_trials
stas
all_amplitudes
sta_amplitudes
t_tests
wilcoxons
sta_sig
sta_sig_nomulti
single_sig
n_targets
target_coords
target_areas
targeted_cells
n_targeted_cells


### Play with whisker stim data

In [ ]:
# Calculate the number of cells that responded both to whisker stim and photostim

# a = np.where(exp_obj.whisker_stim.sta_sig[0])
# b = np.where(exp_obj.photostim_r.sta_sig[0])
# print(a,b)
# sum(1 for i in b[0] if i in a[0])

# Plot whisker stim trials where cells went over 200 dFF

plt.figure(figsize=(5,10))
sta_mean = []

for i, sta in enumerate(exp_obj.whisker_stim.stas[0]):
    if exp_obj.whisker_stim.sta_sig[0][i]:
        plt.plot(sta, alpha=0.2)
        sta_mean.append(sta)
    
        if exp_obj.whisker_stim.sta_amplitudes[0][i] > 200:
            print(exp_obj.whisker_stim.cell_id[0][i])
            
sta_mean = np.mean(sta_mean, axis=0)

plt.plot(sta_mean, 'k', linewidth=2);

plt.axis([0, 55, -10, 300]);

### Play with dataframes from master object

In [4]:
with open(master_path, 'rb') as f:
    master_obj = pickle.load(f)

df = master_obj.df
print(df.columns)

Index(['sheet_name', 'tiff_path', 'stim_type', 'fps', 'n_units', 's1_cells',
       's2_sta', 's2_sta_amp', 's2_sta_std', 's1_sta', 's1_sta_amp',
       's1_sta_std', 'num_s1_cells', 'num_s2_cells',
       'positive_s1_responders_trial', 'negative_s1_responders_trial',
       'positive_s2_responders_trial', 'negative_s2_responders_trial',
       'positive_s1_responders_sta', 'negative_s1_responders_sta',
       'positive_s2_responders_sta', 'negative_s2_responders_sta',
       'positive_s1_responders_sta_nomulti',
       'negative_s1_responders_sta_nomulti',
       'positive_s2_responders_sta_nomulti',
       'negative_s2_responders_sta_nomulti', 'prob_response', 'n_targets',
       'target_cells', 'n_targeted_cells', 'stim_dur', 'stim_freq',
       'target_sta', 'target_sta_amp', 'target_sta_std', 'non_target_sta',
       'non_target_sta_amp', 'non_target_sta_std', 'target_responders_trial',
       'target_responders_trial_sum', 'target_responders',
       'target_responders_sta', 'ta

In [5]:
df

,sheet_name,tiff_path,stim_type,fps,n_units,s1_cells,s2_sta,s2_sta_amp,s2_sta_std,s1_sta,...,non_target_sta_amp,non_target_sta_std,target_responders_trial,target_responders_trial_sum,target_responders,target_responders_sta,target_responders_sta_nomulti,target_sum_dff,num_whisker_targets,euclid_dist
0,2019-06-11_J048,2019-06-11_J048_t-005,pr,36.778513,1158,"[True, False, False, False, False, True, True,...","[9.80126747413565, 3.927418943240566, 3.360084...",-0.232324,"[147.07725087036883, 67.0403847049986, 104.630...","[-2.782787225042642, 7.787154934357725, 15.052...",...,6.261090,"[149.18783537073054, 115.1301149593675, 322.29...","[[False, False, False, False, False, False, Fa...","[53, 47, 41, 50, 46, 54, 37, 46, 42, 39, 48, 4...","[False, False, False, False, False, False, Fal...",76.0,81.0,"[10741.962353545561, 7892.722504675885, 6085.6...",[],"[141.15097870579874, 265.3475912086889, 259.32..."
1,2019-06-11_J048,2019-06-11_J048_t-006,ps,36.778513,1158,"[True, False, False, False, False, True, True,...","[20.304912563219936, -14.587921360146037, -29....",9.614799,"[459.89773538758385, 294.27153669730654, 603.1...","[9.3174515861424, 6.774905047332014, -9.877052...",...,8.410530,"[380.46172722529906, 229.7786406354535, 482.79...","[[False, False, False, False, False, False, Fa...","[22, 23, 17, 13, 34, 25, 18, 15, 16, 8, 11, 11...","[False, False, False, False, False, False, Fal...",44.0,52.0,"[4413.855700959373, 3624.880856790679, 1995.07...",[],"[94.07534181716552, 62.14425029711556, 103.372..."
2,2019-06-13_J052,2019-06-13_J052_t-004,pr,30.974703,421,"[True, False, False, True, False, True, False,...","[0.17207442540440937, 0.6493526007847731, -0.6...",3.184907,"[6.800300698121407, 7.819839175209348, 8.80577...","[0.5663010907424441, 1.6398243861872335, -0.54...",...,3.456112,"[7.988873081125095, 7.2939843284469985, 8.5820...","[[False, False, False, False, False, False, Fa...","[29, 28, 25, 25, 23, 23, 25, 23, 22, 28, 28, 2...","[False, False, False, False, False, False, Fal...",49.0,53.0,"[8444.699997958183, 7753.047400472692, 5826.50...",[],"[61.4687768154425, 181.0380730350064, 100.7203..."
3,2019-06-13_J052,2019-06-13_J052_t-005,ps,30.576645,421,"[True, False, False, True, False, True, False,...","[1.032306694169177, -0.4112868379817025, 0.117...",7.870542,"[21.893180332644157, 9.576685423719116, 13.329...","[1.6190099475039559, -0.17143227235916955, 0.3...",...,6.275847,"[18.129691138396787, 9.010819420558407, 11.238...","[[True, False, False, False, False, False, Fal...","[5, 6, 2, 3, 3, 5, 3, 1, 2, 8, 12, 1, 4, 1, 2,...","[True, False, False, False, False, False, Fals...",18.0,25.0,"[713.5792805626988, 1222.135516781321, 294.670...",[],"[105.44880274332185, 166.44405967171073, 37.06..."
4,2019-06-25_RL048,2019-06-24_RL048_t-005,pr,29.759712,609,"[False, False, False, True, False, False, Fals...","[-7.601071352445154, 16.854814408520472, 17.46...",25.945997,"[148.79972895900133, 331.4012587492237, 322.59...","[-56.52141074058053, 39.21328265658601, -7.695...",...,13.136922,"[552.6662490774399, 494.6481140321293, 283.245...","[[False, False, False, False, False, False, Fa...","[32, 31, 33, 27, 29, 34, 28, 28, 26, 32, 32, 3...","[False, False, False, False, False, False, Fal...",60.0,62.0,"[10115.608731939546, 7395.383273746626, 7197.5...",[],"[99.7569877831636, 92.84449868745297, 92.65612..."
5,2019-06-25_RL048,2019-06-24_RL048_t-006,ps,29.759712,609,"[False, False, False, True, False, False, Fals...","[0.5584547183931289, 0.7058555080627132, 0.993...",2.558476,"[8.084113478476674, 15.775660789263087, 17.309...","[-10.571612151029878, 5.602169824015752, -11.4...",...,3.385677,"[86.53621787114143, 57.1589428296368, 120.4262...","[[False, False, False, False, False, False, Fa...","[10, 5, 9, 9, 6, 5, 10, 8, 6, 5, 7, 9, 5, 7, 6...","[False, False, False, False, False, False, Fal...",21.0,23.0,"[1535.1959129085853, 706.4832036070792, 1313.4...",[],"[115.6896711033444, 75.77492989109263, 113.685..."
6,2019-07-15_RL052,

In [ ]:
# Trying to combine creation of drift_stacks and plotting in to one function

legend = True

pkl_list = [
#             '2019-06-11_J048.pkl',
#             '2019-06-13_J052.pkl',
#             '2019-06-25_RL048.pkl',
#             '2019-07-15_RL052.pkl',
#             '2019-08-14_J058.pkl',
#             '2019-08-14_J059.pkl',
#             '2019-08-15_RL055.pkl',
#             '2019-11-14_J061.pkl',
#             '2019-11-14_J063.pkl',
#             '2019-11-15_J064.pkl',
#             '2019-11-18_J065.pkl',
#             '2020-02-27_RL075.pkl',
            '2020-03-03_RL074.pkl',
#             '2020-03-05_RL070.pkl',
#             '2020-03-05_RL072.pkl',
#             '2020-03-12_RL070.pkl',
#             '2020-03-16_RL074.pkl',
#             '2020-03-19_RL075.pkl',
#             '2020-03-20_RL080.pkl'
           ]

for i,pkl in enumerate(pkl_list):
    pkl_list[i] = os.path.join(qnap_path, 'pkl_files', pkl)

if not pkl_list:
    pkl_list = self.pkl_path

fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(15,10), sharex=True)
labels = []

raw_f_frame = np.empty([len(pkl_list), 8])
norm_f_frame = np.empty([len(pkl_list), 8])

for i,pkl_file in enumerate(pkl_list):

    with open(pkl_file, 'rb') as f:
        exp_obj = pickle.load(f)
        
    obj_list = []
    
    if exp_obj.whisker_stim.n_frames > 0:
        obj_list.append(exp_obj.whisker_stim)
        
    if exp_obj.spont.n_frames > 0:
        obj_list.append(exp_obj.spont)

    obj_list.extend([exp_obj.photostim_r, exp_obj.photostim_s])
        
    size_y = exp_obj.photostim_r.frame_y
    size_x = exp_obj.photostim_r.frame_x
    drift_stack = np.empty([0, size_y, size_x])

    for j,sub_obj in enumerate(obj_list):

        tiff_path = sub_obj.tiff_path

        file_list = os.listdir(tiff_path)
        for file in file_list:
            if '.tif' in file:
                tiff_file = os.path.join(tiff_path, file)

        total_frames = range(0,sub_obj.n_frames) #get the range of frames for this experiment
        start_frames = total_frames[30:1000] # get 970 frames after start
        end_frames = total_frames[-1000:-30] # get 970 frames before end

        stack_start = tf.imread(tiff_file, key=start_frames)
        stack_end = tf.imread(tiff_file, key=end_frames)

        mean_start = np.mean(stack_start, axis=0)
        mean_end = np.mean(stack_end, axis=0)

        drift_stack = np.append(drift_stack, [mean_start, mean_end], axis=0)
                
#         save_path = os.path.join(qnap_path, 'Analysis', 'Drift_check', tiff_path.split('/')[-1])
#         tf.imwrite(save_path + '_drift_stack.tif', drift_stack.astype('int16'))
    
    raw_f = np.mean(drift_stack, axis=(1,2))
    norm_f = raw_f/raw_f[0]
    raw_f_frame[i][:len(raw_f)] = raw_f
    norm_f_frame[i][:len(norm_f)] = norm_f

    ax[0].plot(raw_f)
    ax[1].plot(norm_f)
    labels.append(pkl[:-4])               

plt.xlabel('experiments');
plt.xticks(range(0,8), np.tile(np.array(['start','end']), 4));
ax[0].set_ylabel('raw_f');
ax[1].set_ylabel('norm_f');
ax[1].set_ylim([0.6, 1.2]);       

if legend:
    plt.legend(labels);